In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import iqr
from statsmodels.formula.api import ols
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# Create bar plot
plt.style.use('seaborn-v0_8-poster')
#plt.style.use('seaborn-v0_8')
fontsize = 11
plt.rcParams.update({
    "font.size": fontsize,          # Global font size
    "axes.labelsize": fontsize,     # X and Y axis labels
    "axes.titlesize": fontsize,     # Title (if used)
    "xtick.labelsize": fontsize,    # X-axis tick labels
    "ytick.labelsize": fontsize,    # Y-axis tick labels
    "legend.fontsize": fontsize,    # Legend font size
})

In [2]:
project_dir = '/Users/xiaoqianxiao/projects/NARSAD'
SCR_dir = os.path.join(project_dir, 'EDR')
fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')
plot_dir = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/plots')

In [3]:
def rename_roi(roi):
    cleaned_roi = roi.replace('_mask', '').replace('_mask', '')
    sides = ['lh', 'rh']
    regions = ['hippocampus', 'amygdala', 'vmpfc', 'insula', 'ACC']  # Add more as needed
    
    side = None
    region = None
    for s in sides:
        if s in cleaned_roi:
            side = s
            break
    for r in regions:
        if r in cleaned_roi:
            region = r
            break
    if side and region:
        return f"{side}_{region}"
    else:
        return cleaned_roi

In [4]:
# Define directories
root_dir = '/Users/xiaoqianxiao/projects'
project_name = 'NARSAD'

sub_no_MRI_phase2 = ['N102', 'N208']
sub_no_MRI_phase3 = ['N102', 'N208', 'N120']

SCR_dir = os.path.join(root_dir, project_name, 'EDR')
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')

# Load behavioral data
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')

group_info_df_phase2 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase2)]
group_info_df_phase3 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase3)]

fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
df_fMRI['roi_name'] = df_fMRI['ROI'].apply(rename_roi)
# Define the desired order for 'roi_name'
roi_order = ['lh_ACC', 'rh_ACC', 
             'lh_vmpfc','rh_vmpfc',
             'lh_insula','rh_insula', 
             'lh_hippocampus', 'rh_hippocampus', 
             'lh_amygdala', 'rh_amygdala'
]

df_fMRI['roi_name'] = pd.Categorical(df_fMRI['roi_name'], categories=roi_order, ordered=True)

df_fMRI_phase2 = df_fMRI.loc[df_fMRI['Task']=='phase2'].copy()
df_fMRI_phase3 = df_fMRI.loc[df_fMRI['Task']=='phase3'].copy()

In [5]:
df_fMRI_phase2['subID'] = df_fMRI_phase2['Subject'].apply(lambda x: group_info_df_phase2.iloc[x - 1]['subID'])
df_fMRI_phase3['subID'] = df_fMRI_phase3['Subject'].apply(lambda x: group_info_df_phase3.iloc[x - 1]['subID'])
df_phase2 = df_fMRI_phase2.merge(group_info_df_phase2, how='left', left_on='subID', right_on='subID')
df_phase2 = df_phase2[df_phase2['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)
df_phase3 = df_fMRI_phase3.merge(group_info_df_phase3, how='left', left_on='subID', right_on='subID')
df_phase3 = df_phase3[df_phase3['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)

In [6]:
df_current = df_phase2
contrasts_of_interest = [1,3]
contrasts_CSS = contrasts_of_interest[0]
contrasts_CSR = contrasts_of_interest[1]

In [7]:
results = []
from scipy.stats import ttest_rel
for roi in df_current['roi_name'].unique():
    for group in df_current['group'].unique():
        for gender in df_current['Gender'].unique():
            for drug in df_current['Drug'].unique():
                df_tmp = df_current.loc[(df_current['roi_name'] == roi) & 
                                        (df_current['group'] == group) & 
                                        (df_current['Gender'] == gender) &
                                        (df_current['Drug'] == drug) &
                                        (df_current['Contrast'].isin(contrasts_of_interest))].copy()
                df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
                t_statistic, p_value = stats.ttest_rel(df_pivot[[contrasts_CSS]], df_pivot[[contrasts_CSR]])
                results.append({
                'roi': roi,
                'Group': group,
                'Drug': drug,
                'Gender': gender,
                't_stat': t_statistic[0],
                'p_value': p_value[0],
                'n': len(df_pivot)  # Sample size
            })
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

,roi,Group,Drug,Gender,t_stat,p_value,n,p_value_corrected
12,rh_ACC,Controls,Placebo,Female,3.142166,0.007788,14,0.623057
0,lh_hippocampus,Patients,Placebo,Female,-0.605616,0.556051,13,1.000000
57,rh_insula,Patients,Oxytocin,Female,-0.897598,0.383581,16,1.000000
56,rh_insula,Patients,Placebo,Female,0.695904,0.499746,13,1.000000
55,rh_amygdala,Controls,Oxytocin,Male,1.368942,0.198321,12,1.000000
...,...,...,...,...,...,...,...,...
24,lh_ACC,Patients,Placebo,Female,-0.430922,0.674164,13,1.000000
23,lh_amygdala,Controls,Oxytocin,Male,-0.014077,0.989020,12,1.000000
22,lh_amygdala,Controls,Placebo,Male,-1.072422,0.303035,14,1.000000
29,lh_ACC,Controls,Oxytocin,Female,1.859879,0.089832,12,1.000000


hypothesis: brain activations should be different between CSS and CSR if extinction affect.\
For patients:\
lh-insula: female-oxytocin [lh-insula showed no differences for controls  and patient males(with/without oxytocin), but showed greater activation in CSS(than CSR) with oxytocin in female patients]\
rh-amygdala (marginal): male-oxytocin [rh-amygdala showed no differences for controls  and patient females(with/without oxytocin), but showed smaller activation in CSS with oxytocin in male patients]\
rh-vmpfc: male-placebo [rh-vmpfc showed no differences for controls  and patient females(with/without oxytocin), but showed greater activation in CSR in placebo in male patients]\
For controls: \
rh-ACC, rh-amy and rh-vmpfc(marginal): female-placebo\
lh-acc:male-placebo

In [8]:
contrasts_of_interest = [5]
# extinction
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                 -0.039
Method:                 Least Squares   F-statistic:                    0.5825
Date:                Mon, 31 Mar 2025   Prob (F-statistic):              0.808
Time:                        12:20:59   Log-Likelihood:                -140.48
No. Observations:                 101   AIC:                             301.0
Df Residuals:                      91   BIC:                             327.1
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

,Condition,Effect,p_value
12,rh_ACC,C(Drug)[T.Placebo],0.003258
68,rh_amygdala,Avoidance_centered,0.026544
14,rh_ACC,C(group)[T.Patients]:C(Drug)[T.Placebo],0.041556
28,lh_amygdala,Avoidance_centered,0.053587
16,rh_ACC,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.055320
...,...,...,...
82,lh_vmpfc,C(Drug)[T.Placebo],0.939932
83,lh_vmpfc,C(Gender)[T.Male],0.955511
19,rh_ACC,demo_age,0.959436
47,rh_vmpfc,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.968514


In [9]:
# extinction
contrasts_of_interest = [1,3]
contrasts_CSS = contrasts_of_interest[0]
contrasts_CSR = contrasts_of_interest[1]
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                 -0.039
Method:                 Least Squares   F-statistic:                    0.5825
Date:                Mon, 31 Mar 2025   Prob (F-statistic):              0.808
Time:                        12:21:24   Log-Likelihood:                -140.48
No. Observations:                 101   AIC:                             301.0
Df Residuals:                      91   BIC:                             327.1
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

,Condition,Effect,p_value
12,rh_ACC,C(Drug)[T.Placebo],0.003258
68,rh_amygdala,Avoidance_centered,0.026544
14,rh_ACC,C(group)[T.Patients]:C(Drug)[T.Placebo],0.041556
28,lh_amygdala,Avoidance_centered,0.053587
16,rh_ACC,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.055320
...,...,...,...
82,lh_vmpfc,C(Drug)[T.Placebo],0.939932
83,lh_vmpfc,C(Gender)[T.Male],0.955511
19,rh_ACC,demo_age,0.959436
47,rh_vmpfc,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.968514


for the entire dataset \
rh-ACC: Drug effect, group * drug and group * gender(marginal) \
Avoidance rh-amy (lh-amy, marginal)

In [201]:
# extinction
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender)", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                 -0.031
Method:                 Least Squares   F-statistic:                    0.5708
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.778
Time:                        12:46:51   Log-Likelihood:                -141.19
No. Observations:                 101   AIC:                             298.4
Df Residuals:                      93   BIC:                             319.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

,Condition,Effect,p_value
10,rh_ACC,C(Drug)[T.Placebo],0.002746
12,rh_ACC,C(group)[T.Patients]:C(Drug)[T.Placebo],0.038878
14,rh_ACC,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.047652
53,rh_amygdala,C(group)[T.Patients]:C(Gender)[T.Male],0.078031
45,lh_insula,C(group)[T.Patients]:C(Gender)[T.Male],0.088105
...,...,...,...
54,rh_amygdala,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.958330
71,lh_vmpfc,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.972489
66,lh_vmpfc,C(Drug)[T.Placebo],0.979234
35,rh_vmpfc,C(Gender)[T.Male],0.983534


In [210]:
# extinction in only placebo, healthy females
results = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') &
                        (df_current['group'] == 'Controls') & 
                        (df_current['Gender'] == 'Female') & 
                        (df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
    t_statistic, p_value = stats.ttest_rel(df_pivot[[contrasts_CSS]], df_pivot[[contrasts_CSR]])
    results.append({
    'roi': roi,
    't_stat': t_statistic[0],
    'p_value': p_value[0],
    'n': len(df_pivot)  # Sample size
})
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

extinction results for lh_hippocampus
extinction results for rh_ACC
extinction results for lh_amygdala
extinction results for lh_ACC
extinction results for rh_vmpfc
extinction results for lh_insula
extinction results for rh_amygdala
extinction results for rh_insula
extinction results for lh_vmpfc
extinction results for rh_hippocampus


,roi,t_stat,p_value,n,p_value_corrected
1,rh_ACC,3.142166,0.007788,14,0.077882
6,rh_amygdala,-2.342213,0.035741,14,0.357409
4,rh_vmpfc,2.070127,0.058916,14,0.589162
5,lh_insula,1.872897,0.083741,14,0.837414
0,lh_hippocampus,0.833921,0.419385,14,1.000000
2,lh_amygdala,0.815089,0.429702,14,1.000000
3,lh_ACC,1.000755,0.335210,14,1.000000
7,rh_insula,0.819481,0.427281,14,1.000000
8,lh_vmpfc,0.489049,0.632955,14,1.000000
9,rh_hippocampus,-0.629778,0.539751,14,1.000000


In [187]:
# extinction in only placebo
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI     
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'ROI': roi,
            'Effect': effect_idx,
            'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
        })
    
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                 -0.080
Method:                 Least Squares   F-statistic:                    0.2783
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.923
Time:                        11:54:54   Log-Likelihood:                -70.169
No. Observations:                  50   AIC:                             152.3
Df Residuals:                      44   BIC:                             163.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

,ROI,Effect,b,SE,t,p,CI_lower,CI_upper
8,rh_ACC,C(Gender)[T.Male],1.076997,0.357258,3.014618,0.004260,0.356990,1.797004
7,rh_ACC,C(group)[T.Patients],1.009939,0.379976,2.657901,0.010919,0.244147,1.775731
37,rh_amygdala,C(group)[T.Patients],-0.924006,0.418190,-2.209539,0.032387,-1.766812,-0.081200
20,lh_ACC,C(Gender)[T.Male],0.799086,0.394826,2.023894,0.049080,0.003367,1.594805
25,rh_vmpfc,C(group)[T.Patients],0.731250,0.381357,1.917494,0.061681,-0.037325,1.499825
28,rh_vmpfc,Avoidance_centered,-0.012439,0.006647,-1.871473,0.067937,-0.025834,0.000956
47,rh_insula,demo_age,0.040199,0.021902,1.835406,0.073210,-0.003942,0.084340
26,rh_vmpfc,C(Gender)[T.Male],0.529319,0.358557,1.476248,0.146999,-0.193305,1.251942
40,rh_amygdala,Avoidance_centered,0.010271,0.007289,1.409147,0.165820,-0.004418,0.024960
16,lh_amygdala,Avoidance_centered,-0.010091,0.007326,-1.377345,0.175372,-0.024855,0.004674


In placebo. \
Gender in bi-ACC; \
Group in rh-ACC, rh-AMY, rh-vmpfc (marginal); 

In [202]:
# extinction in each group
results_list = []
drug_results_list = []
for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {group}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                          (df_current['group'] == group) & 
                          (df_current['Contrast'].isin(contrasts_of_interest))].copy()
        
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                        columns='CS_Type', 
                        values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
        #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        
        combined_col = f'{roi}_{group}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Effect': effect_idx,
                'p_value': p_value
            })
        conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
        for effect_idx in result_ols.params.index:
            if effect_idx == 'C(Drug)[T.Placebo]': 
                drug_results_list.append({
                    'Effect': effect_idx,
                    'group': group,
                    'ROI': roi,
                    'b': result_ols.params[effect_idx],  # Coefficient
                    'SE': result_ols.bse[effect_idx],    # Standard error
                    't': result_ols.tvalues[effect_idx], # t-statistic
                    'p': result_ols.pvalues[effect_idx],  # p-value
                    'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                    'CI_upper': conf_int.loc[effect_idx, 1]
                })
    
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('p', ascending=True)
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Patients
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.085
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                    0.7975
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.558
Time:                        12:49:15   Log-Likelihood:                -67.355
No. Observations:                  49   AIC:                             146.7
Df Residuals:                      43   BIC:                             158.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

,Condition,Effect,p_value
67,rh_ACC_Controls,C(Drug)[T.Placebo],0.008632
94,lh_insula_Controls,Avoidance_centered,0.015099
32,lh_insula_Patients,C(Gender)[T.Male],0.023231
40,rh_amygdala_Patients,Avoidance_centered,0.033443
52,lh_vmpfc_Patients,Avoidance_centered,0.041689
...,...,...,...
92,lh_insula_Controls,C(Gender)[T.Male],0.928580
116,rh_hippocampus_Controls,C(Gender)[T.Male],0.942887
68,rh_ACC_Controls,C(Gender)[T.Male],0.952104
82,lh_ACC_Controls,Avoidance_centered,0.955711


in group specific dataset, \
for Patients: gender in lh-insula \
        avoidance in rh-amy and lh-vmpfc
for Controls: drug effect in rh_ACC, and avoidance in lh-insula

In [203]:
import pandas as pd
from scipy.stats import zscore, ttest_ind

desired_comparisons = [
    'Female_Oxytocin vs. Female_Placebo',
    'Male_Oxytocin vs. Male_Placebo'
]

t_test_results_list = []

for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                           (df_current['group'] == group) & 
                           (df_current['Contrast'].isin(contrasts_of_interest))].copy()
        
        df_tmp.loc[df_tmp['Contrast'] == contrasts_CSS, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast'] == contrasts_CSR, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID', 'Drug', 'Gender', 'group', 'Avoidance', 'demo_age'], 
                          columns='CS_Type', values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])  # Offset of 1 assumed intentional
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
        
        group_labels = df['Gender'].astype(str) + "_" + df['Drug'].astype(str)
        
        for comparison in desired_comparisons:
            g1, g2 = comparison.split(' vs. ')
            if g1 in group_labels.values and g2 in group_labels.values:  # Check if both groups exist
                group1_data = df[group_labels == g1]['z_fMRI']
                group2_data = df[group_labels == g2]['z_fMRI']
                
                t_stat, p_value = ttest_ind(group1_data, group2_data, nan_policy='omit')
                
                mean_diff = group1_data.mean() - group2_data.mean()
                se = ((group1_data.var() / group1_data.count()) + (group2_data.var() / group2_data.count())) ** 0.5
                
                t_test_results_list.append({
                    'ROI': roi,
                    'Comparison': comparison,
                    'Mean_Diff': mean_diff,
                    'SE': se,
                    't': t_stat,
                    'p': p_value
                })

# Create DataFrame with all results
df_t_test = pd.DataFrame(t_test_results_list)
df_t_test = df_t_test.sort_values('p', ascending=True)  # Sort by p-value

In [204]:
results_list = []
# extinction, in each gender
for g in df_current['Gender'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {g}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Gender'] == g) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(group) * C(Drug) + Avoidance_centered + demo_age", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        combined_col = f'{roi}_{g}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Gender': g,
                'Effect': effect_idx,
                'p_value': p_value
            })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                 -0.026
Method:                 Least Squares   F-statistic:                    0.7266
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.607
Time:                        12:50:38   Log-Likelihood:                -76.075
No. Observations:                  55   AIC:                             164.1
Df Residuals:                      49   BIC:                             176.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

,Condition,Gender,Effect,p_value
8,rh_ACC_Female,Female,C(Drug)[T.Placebo],0.005497
59,rh_hippocampus_Female,Female,demo_age,0.026297
89,rh_vmpfc_Male,Male,demo_age,0.035092
52,lh_vmpfc_Female,Female,Avoidance_centered,0.038020
99,rh_amygdala_Male,Male,C(group)[T.Patients]:C(Drug)[T.Placebo],0.038134
...,...,...,...,...
58,rh_hippocampus_Female,Female,Avoidance_centered,0.951606
94,lh_insula_Male,Male,Avoidance_centered,0.958607
20,lh_ACC_Female,Female,C(Drug)[T.Placebo],0.974131
117,rh_hippocampus_Male,Male,C(group)[T.Patients]:C(Drug)[T.Placebo],0.980259


in Gender specific data subset, \
for Female: \
    drug effect in rh-acc, \
    group * drug in rh-acc
    age in rh-hip \
    avoidance in lh-vmpfc \
for Male: \
    age in rh-vmpfc \
    group * drun in rh-amy \
    avoidance(marginal) in lh_amygdala

In [205]:
import pandas as pd 
from statsmodels.formula.api import ols
from scipy.stats import zscore

results_list = []
for g in df_current['Gender'].unique():
    for group in df_current['group'].unique():
        for roi in df_current['roi_name'].unique():
            print(f'extinction results for {roi} {g}')
            df_tmp = df_current[(df_current['roi_name'] == roi) & 
                              (df_current['Gender'] == g) & 
                              (df_current['group'] == group) & 
                              (df_current['Contrast'].isin(contrasts_of_interest))].copy()
            
            df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
            df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
            
            df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                            columns='CS_Type', 
                            values='Mean_COPE_Value').reset_index()
            
            df['fMRI'] = df['CSR'] - df['CSS']
            df['z_fMRI'] = 1 + zscore(df['fMRI'])
            df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
            df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
            
            df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
            
            model_ols = ols("z_fMRI ~ C(Drug) + Avoidance_centered + demo_age", data=df)
            
            #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
            result_ols = model_ols.fit()
            print(result_ols.summary())
            
            combined_col = f'{roi}_{group}_{g}'
            
            for effect_idx in result_ols.pvalues.index:
                p_value = result_ols.pvalues[effect_idx]
                results_list.append({
                    'Gender': g,
                    'Group': group,
                    'Condition': combined_col,
                    'Effect': effect_idx,
                    'p_value': p_value
                })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                 -0.040
Method:                 Least Squares   F-statistic:                    0.6420
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.595
Time:                        12:51:22   Log-Likelihood:                -40.073
No. Observations:                  29   AIC:                             88.15
Df Residuals:                      25   BIC:                             93.62
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

,Gender,Group,Condition,Effect,p_value
154,Male,Controls,lh_vmpfc_Controls_Male,Avoidance_centered,0.011500
45,Female,Controls,rh_ACC_Controls_Female,C(Drug)[T.Placebo],0.025063
39,Female,Patients,rh_hippocampus_Patients_Female,demo_age,0.031116
138,Male,Controls,rh_vmpfc_Controls_Male,Avoidance_centered,0.048475
75,Female,Controls,lh_vmpfc_Controls_Female,demo_age,0.054956
...,...,...,...,...,...
118,Male,Patients,rh_hippocampus_Patients_Male,Avoidance_centered,0.975212
125,Male,Controls,rh_ACC_Controls_Male,C(Drug)[T.Placebo],0.984399
53,Female,Controls,lh_ACC_Controls_Female,C(Drug)[T.Placebo],0.985640
147,Male,Controls,rh_amygdala_Controls_Male,demo_age,0.988299


in group * gender specific data subset:\
avoidance in lh-vmpfc for Male controls \
drug in rh-acc for Female controls \
age in rh-hip for Female patients \
avoidance in rh-vmpfc for Male controls \

In [188]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                col='group',
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_gender_group.png', bbox_inches='tight')
plt.close()
#plt.show()

In [189]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_gender_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [166]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='group', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_group_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [173]:
for roi in df_current['roi_name'].unique():
    print(f'{roi}')
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast'] == contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast'] == contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    
    df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    
    g = sns.catplot(x='Drug', y='z_fMRI', hue='CS_Type',
                    order=['Placebo', 'Oxytocin'], 
                    col='group', 
                    row='Gender', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    
    g.set_axis_labels('', 'z_fMRI')
    #g.legend.set_title('')
    if g.legend is not None:
        g.legend.set_title('')
    else:
        print(f"Warning: No legend generated for {roi_name}")    
    y_min = df['z_fMRI'].min() - 0.5
    y_max = df['z_fMRI'].max() + 0.5
    g.set(ylim=(0, 2.2))
    
    g.figure.suptitle(f'{roi} in Extinction', fontsize=22, y=1.01)  # Add title above plot
    
    g.figure.set_dpi(300)
    plt.savefig(f'{plot_dir}/extinction_plot_{roi}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

lh_hippocampus
rh_ACC
lh_amygdala
lh_ACC
rh_vmpfc
lh_insula
rh_amygdala
rh_insula
lh_vmpfc
rh_hippocampus
